<h1 align=center>Convolutional Neural Networks Training Notebook</h1>
<hr>

### Dataset

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

/Users/wisnumulya/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/wisnumulya/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/wisnumulya/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Use

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### Weights and Biases

In [2]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

### Networks Functions

In [3]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

### Session

In [4]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch  1, Batch   1 -Loss: 65693.3672 Validation Accuracy: 0.132812
Epoch  1, Batch   2 -Loss: 56736.9531 Validation Accuracy: 0.121094
Epoch  1, Batch   3 -Loss: 50502.2188 Validation Accuracy: 0.113281
Epoch  1, Batch   4 -Loss: 42193.9922 Validation Accuracy: 0.152344
Epoch  1, Batch   5 -Loss: 42668.4258 Validation Accuracy: 0.132812
Epoch  1, Batch   6 -Loss: 40521.0000 Validation Accuracy: 0.148438
Epoch  1, Batch   7 -Loss: 30988.0039 Validation Accuracy: 0.171875
Epoch  1, Batch   8 -Loss: 29794.3867 Validation Accuracy: 0.222656
Epoch  1, Batch   9 -Loss: 29683.5977 Validation Accuracy: 0.238281
Epoch  1, Batch  10 -Loss: 28338.3027 Validation Accuracy: 0.234375
Epoch  1, Batc

Epoch  1, Batch 112 -Loss:  4911.8149 Validation Accuracy: 0.589844
Epoch  1, Batch 113 -Loss:  3658.0364 Validation Accuracy: 0.593750
Epoch  1, Batch 114 -Loss:  3878.5417 Validation Accuracy: 0.589844
Epoch  1, Batch 115 -Loss:  4174.8364 Validation Accuracy: 0.605469
Epoch  1, Batch 116 -Loss:  3155.3838 Validation Accuracy: 0.597656
Epoch  1, Batch 117 -Loss:  4239.6914 Validation Accuracy: 0.597656
Epoch  1, Batch 118 -Loss:  3769.6768 Validation Accuracy: 0.613281
Epoch  1, Batch 119 -Loss:  3972.1147 Validation Accuracy: 0.597656
Epoch  1, Batch 120 -Loss:  3472.0527 Validation Accuracy: 0.597656
Epoch  1, Batch 121 -Loss:  3323.9365 Validation Accuracy: 0.601562
Epoch  1, Batch 122 -Loss:  3328.8384 Validation Accuracy: 0.605469
Epoch  1, Batch 123 -Loss:  3615.0181 Validation Accuracy: 0.609375
Epoch  1, Batch 124 -Loss:  3695.5830 Validation Accuracy: 0.605469
Epoch  1, Batch 125 -Loss:  3474.7166 Validation Accuracy: 0.613281
Epoch  1, Batch 126 -Loss:  3210.9844 Validation

Epoch  1, Batch 233 -Loss:  1722.2140 Validation Accuracy: 0.718750
Epoch  1, Batch 234 -Loss:  2093.3091 Validation Accuracy: 0.714844
Epoch  1, Batch 235 -Loss:  1473.5361 Validation Accuracy: 0.714844
Epoch  1, Batch 236 -Loss:  2415.7192 Validation Accuracy: 0.714844
Epoch  1, Batch 237 -Loss:  2093.3789 Validation Accuracy: 0.710938
Epoch  1, Batch 238 -Loss:  1894.6711 Validation Accuracy: 0.710938
Epoch  1, Batch 239 -Loss:  2791.1846 Validation Accuracy: 0.710938
Epoch  1, Batch 240 -Loss:  2322.0542 Validation Accuracy: 0.714844
Epoch  1, Batch 241 -Loss:  1984.9214 Validation Accuracy: 0.714844
Epoch  1, Batch 242 -Loss:  2264.4224 Validation Accuracy: 0.714844
Epoch  1, Batch 243 -Loss:  1882.8433 Validation Accuracy: 0.714844
Epoch  1, Batch 244 -Loss:  2093.2600 Validation Accuracy: 0.718750
Epoch  1, Batch 245 -Loss:  2122.5803 Validation Accuracy: 0.718750
Epoch  1, Batch 246 -Loss:  1892.0542 Validation Accuracy: 0.718750
Epoch  1, Batch 247 -Loss:  2574.4673 Validation

Epoch  1, Batch 354 -Loss:  1763.6011 Validation Accuracy: 0.734375
Epoch  1, Batch 355 -Loss:  1545.0219 Validation Accuracy: 0.738281
Epoch  1, Batch 356 -Loss:  1635.9363 Validation Accuracy: 0.738281
Epoch  1, Batch 357 -Loss:  1648.7139 Validation Accuracy: 0.738281
Epoch  1, Batch 358 -Loss:  1477.8755 Validation Accuracy: 0.734375
Epoch  1, Batch 359 -Loss:  1497.2627 Validation Accuracy: 0.742188
Epoch  1, Batch 360 -Loss:  1716.8149 Validation Accuracy: 0.746094
Epoch  1, Batch 361 -Loss:  1596.2688 Validation Accuracy: 0.746094
Epoch  1, Batch 362 -Loss:  2109.7075 Validation Accuracy: 0.734375
Epoch  1, Batch 363 -Loss:  1942.4127 Validation Accuracy: 0.742188
Epoch  1, Batch 364 -Loss:  1069.5637 Validation Accuracy: 0.734375
Epoch  1, Batch 365 -Loss:  1526.3174 Validation Accuracy: 0.738281
Epoch  1, Batch 366 -Loss:  1506.4996 Validation Accuracy: 0.742188
Epoch  1, Batch 367 -Loss:  1805.7623 Validation Accuracy: 0.734375
Epoch  1, Batch 368 -Loss:  1687.1606 Validation

Epoch  2, Batch  46 -Loss:  1244.4813 Validation Accuracy: 0.722656
Epoch  2, Batch  47 -Loss:  1321.9464 Validation Accuracy: 0.722656
Epoch  2, Batch  48 -Loss:  1419.0745 Validation Accuracy: 0.718750
Epoch  2, Batch  49 -Loss:   781.0703 Validation Accuracy: 0.722656
Epoch  2, Batch  50 -Loss:   803.5604 Validation Accuracy: 0.722656
Epoch  2, Batch  51 -Loss:  1351.3936 Validation Accuracy: 0.718750
Epoch  2, Batch  52 -Loss:  1142.5205 Validation Accuracy: 0.718750
Epoch  2, Batch  53 -Loss:  1345.4984 Validation Accuracy: 0.718750
Epoch  2, Batch  54 -Loss:  1162.8113 Validation Accuracy: 0.718750
Epoch  2, Batch  55 -Loss:  1229.1074 Validation Accuracy: 0.718750
Epoch  2, Batch  56 -Loss:  1040.0847 Validation Accuracy: 0.718750
Epoch  2, Batch  57 -Loss:   995.6638 Validation Accuracy: 0.718750
Epoch  2, Batch  58 -Loss:  1639.2322 Validation Accuracy: 0.714844
Epoch  2, Batch  59 -Loss:  1202.6150 Validation Accuracy: 0.714844
Epoch  2, Batch  60 -Loss:  1134.5737 Validation

...

Epoch  9, Batch 310 -Loss:   146.7404 Validation Accuracy: 0.753906
Epoch  9, Batch 311 -Loss:   142.1389 Validation Accuracy: 0.746094
Epoch  9, Batch 312 -Loss:   261.1216 Validation Accuracy: 0.742188
Epoch  9, Batch 313 -Loss:   261.1365 Validation Accuracy: 0.753906
Epoch  9, Batch 314 -Loss:   157.2637 Validation Accuracy: 0.746094
Epoch  9, Batch 315 -Loss:   197.9401 Validation Accuracy: 0.746094
Epoch  9, Batch 316 -Loss:   220.6566 Validation Accuracy: 0.750000
Epoch  9, Batch 317 -Loss:   215.7748 Validation Accuracy: 0.746094
Epoch  9, Batch 318 -Loss:   162.1978 Validation Accuracy: 0.734375
Epoch  9, Batch 319 -Loss:   160.8160 Validation Accuracy: 0.730469
Epoch  9, Batch 320 -Loss:   123.4856 Validation Accuracy: 0.746094
Epoch  9, Batch 321 -Loss:   230.9026 Validation Accuracy: 0.742188
Epoch  9, Batch 322 -Loss:   174.7076 Validation Accuracy: 0.738281
Epoch  9, Batch 323 -Loss:   153.2836 Validation Accuracy: 0.738281
Epoch  9, Batch 324 -Loss:   243.4621 Validation

Epoch 10, Batch   2 -Loss:   124.4820 Validation Accuracy: 0.746094
Epoch 10, Batch   3 -Loss:   119.6329 Validation Accuracy: 0.746094
Epoch 10, Batch   4 -Loss:   185.8301 Validation Accuracy: 0.742188
Epoch 10, Batch   5 -Loss:   123.6557 Validation Accuracy: 0.746094
Epoch 10, Batch   6 -Loss:   121.7771 Validation Accuracy: 0.742188
Epoch 10, Batch   7 -Loss:   234.8684 Validation Accuracy: 0.742188
Epoch 10, Batch   8 -Loss:   202.9845 Validation Accuracy: 0.742188
Epoch 10, Batch   9 -Loss:   185.6369 Validation Accuracy: 0.746094
Epoch 10, Batch  10 -Loss:   157.7182 Validation Accuracy: 0.742188
Epoch 10, Batch  11 -Loss:   176.9906 Validation Accuracy: 0.746094
Epoch 10, Batch  12 -Loss:   156.1318 Validation Accuracy: 0.738281
Epoch 10, Batch  13 -Loss:   151.5500 Validation Accuracy: 0.734375
Epoch 10, Batch  14 -Loss:   265.7573 Validation Accuracy: 0.738281
Epoch 10, Batch  15 -Loss:   186.0797 Validation Accuracy: 0.734375
Epoch 10, Batch  16 -Loss:   212.1310 Validation

Epoch 10, Batch 123 -Loss:   222.3622 Validation Accuracy: 0.738281
Epoch 10, Batch 124 -Loss:   166.1219 Validation Accuracy: 0.738281
Epoch 10, Batch 125 -Loss:   169.1390 Validation Accuracy: 0.738281
Epoch 10, Batch 126 -Loss:   150.3458 Validation Accuracy: 0.726562
Epoch 10, Batch 127 -Loss:   163.4863 Validation Accuracy: 0.730469
Epoch 10, Batch 128 -Loss:   157.7620 Validation Accuracy: 0.730469
Epoch 10, Batch 129 -Loss:   206.0804 Validation Accuracy: 0.730469
Epoch 10, Batch 130 -Loss:   158.7642 Validation Accuracy: 0.726562
Epoch 10, Batch 131 -Loss:   170.9916 Validation Accuracy: 0.726562
Epoch 10, Batch 132 -Loss:   192.0539 Validation Accuracy: 0.726562
Epoch 10, Batch 133 -Loss:   144.1466 Validation Accuracy: 0.722656
Epoch 10, Batch 134 -Loss:   182.2277 Validation Accuracy: 0.722656
Epoch 10, Batch 135 -Loss:   201.6918 Validation Accuracy: 0.718750
Epoch 10, Batch 136 -Loss:   158.2433 Validation Accuracy: 0.726562
Epoch 10, Batch 137 -Loss:   263.6180 Validation

Epoch 10, Batch 244 -Loss:   169.1731 Validation Accuracy: 0.738281
Epoch 10, Batch 245 -Loss:   184.9587 Validation Accuracy: 0.738281
Epoch 10, Batch 246 -Loss:   177.1998 Validation Accuracy: 0.757812
Epoch 10, Batch 247 -Loss:   147.2743 Validation Accuracy: 0.750000
Epoch 10, Batch 248 -Loss:   175.1747 Validation Accuracy: 0.742188
Epoch 10, Batch 249 -Loss:   177.6335 Validation Accuracy: 0.742188
Epoch 10, Batch 250 -Loss:   151.9487 Validation Accuracy: 0.738281
Epoch 10, Batch 251 -Loss:   174.3582 Validation Accuracy: 0.742188
Epoch 10, Batch 252 -Loss:   126.5639 Validation Accuracy: 0.734375
Epoch 10, Batch 253 -Loss:   176.8715 Validation Accuracy: 0.738281
Epoch 10, Batch 254 -Loss:   227.5772 Validation Accuracy: 0.742188
Epoch 10, Batch 255 -Loss:   131.5691 Validation Accuracy: 0.738281
Epoch 10, Batch 256 -Loss:   175.5975 Validation Accuracy: 0.738281
Epoch 10, Batch 257 -Loss:   184.9441 Validation Accuracy: 0.738281
Epoch 10, Batch 258 -Loss:   169.2303 Validation

Epoch 10, Batch 365 -Loss:    95.2317 Validation Accuracy: 0.734375
Epoch 10, Batch 366 -Loss:   171.4749 Validation Accuracy: 0.730469
Epoch 10, Batch 367 -Loss:   191.2875 Validation Accuracy: 0.734375
Epoch 10, Batch 368 -Loss:   154.4231 Validation Accuracy: 0.734375
Epoch 10, Batch 369 -Loss:   161.7970 Validation Accuracy: 0.726562
Epoch 10, Batch 370 -Loss:   220.9362 Validation Accuracy: 0.722656
Epoch 10, Batch 371 -Loss:   157.0656 Validation Accuracy: 0.718750
Epoch 10, Batch 372 -Loss:   165.9610 Validation Accuracy: 0.710938
Epoch 10, Batch 373 -Loss:   230.5935 Validation Accuracy: 0.722656
Epoch 10, Batch 374 -Loss:   184.1853 Validation Accuracy: 0.718750
Epoch 10, Batch 375 -Loss:   194.5607 Validation Accuracy: 0.718750
Epoch 10, Batch 376 -Loss:   139.5825 Validation Accuracy: 0.718750
Epoch 10, Batch 377 -Loss:   189.1857 Validation Accuracy: 0.722656
Epoch 10, Batch 378 -Loss:   210.4603 Validation Accuracy: 0.726562
Epoch 10, Batch 379 -Loss:   178.7423 Validation